# Increasing Insight into Performance Profiles using Python DataViz Tools

NAG are a key member of the [POP (Performance Optimisation and Productivity](https://pop-coe.eu) Centre of Excellence, providing free of charge software optimisation assistance to European science and industry. A key part of POP's mission is to improve the impact and accessibility of the POP tools, allowing users to more quickly gain deeper insights into the behaviour of their parallel code.

Displaying profiling results clearly and intuitively is key to enabling understanding, and so NAG have developed a data exploration tool named PyPOP to enable rapid visualisation and exploration.  PyPOP is designed to be used within Jupyter notebooks the Pandas and Bokeh libraries for data management and display. This encourages the use of so called "literate programming" where data analysis and visualisations can be combined with free text, allowing creation, description and discussion of the analysis in a single workbook.

In [1]:
import os
import sys

# Import the functions needed to calculate the standard MPI metrics
from pypop.traceset import TraceSet
from pypop.metrics import OpenMP_Metrics
from pypop.prv import PRV

import matplotlib.colors as mc

# If Paramedir or Dimemas are not on your PATH, you can set their directories below
from pypop.config import set_dimemas_path, set_paramedir_path
#set_paramedir_path('/home/telemin/downloads/wxparaver-4.8.2-Linux_x86_64/bin')
#set_dimemas_path('/home/telemin/downloads/dimemas-5.4.1-Linux_x86_64/bin')

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import HoverTool
from bokeh.palettes import diverging_palette, all_palettes
from bokeh.transform import linear_cmap
from bokeh.colors import RGB

import numpy as np
import pandas as pd
output_notebook(hide_banner=True)

In [2]:
trace_directory = '../openmp/imagemagick_example_traces/'

# Make a list of the tracefiles that we want
trace_files = [os.path.join(trace_directory, f) for f in os.listdir(trace_directory) if f.endswith(('.prv','.prv.gz'))]

# Use paramedir to calculate the statistics
statistics = TraceSet(trace_files)

metrics = OpenMP_Metrics(statistics.by_threads_per_process())


A Jupyter Widget

In [3]:
def approx_string_em_width(string):
    """Give the approximate string width in 'em'
    
    Basically assume 'w' and 'm' have em width and everything else has
    a width of 0.6*em
    """
    
    return 0.6*len(string) + 0.4*sum(string.count(x) for x in ['w','m','~'])

In [4]:
bad_thres = 0.5
good_thres = 0.8

cmap_points = [
    (0.0, (0.690, 0.074, 0.074)),
    (bad_thres, (0.690, 0.074, 0.074)),
    (good_thres - 1e-20, (0.992, 0.910, 0.910)),
    (good_thres, (0.910, 0.992, 0.910)),
    (1.0, (0.074, 0.690, 0.074)),
]

metric_cmap = mc.LinearSegmentedColormap.from_list(
    "POP_Metrics", colors=cmap_points, N=256, gamma=1
)


pt_to_px = 96/72
fontsize=16

cell_text = "~~~~~~~~~~~~~~~~~~"

font_px = fontsize*pt_to_px
cell_height = font_px*2.2

border_pad = 20 #px

left_pad = font_px/2
right_pad = font_px/3

nrows = len(metrics.metrics)+1
ncols = len(metrics.metric_data.index)

metric_column_width = 0
row_locs = np.linspace(0,-cell_height*(nrows-1),nrows)
metric_column_text = ['']
metric_descriptions = ['The number of processes the application was run on']
for i, metric in enumerate(metrics.metrics):
    metric_column_text.append(metric.key)
    metric_descriptions.append(metric.description)
    metric_column_width = max(metric_column_width, approx_string_em_width(metric.key))

descriptions = metric_column_text
descriptions[0] = 'Number of Processes'



metric_column_width = metric_column_width*font_px + left_pad + right_pad
value_column_width = approx_string_em_width('0.00')*font_px + left_pad + right_pad

plot_width = int(metric_column_width + ncols*value_column_width) + border_pad # 20 px padding at edge
plot_height = int(cell_height*nrows) + border_pad #px

tooltip_template = """
<div style="width:300px;color:#060684;font-family:sans;font-weight:bold;">
@short_desc
</div>
<div style="width:300px;padding-top:5px;">
@long_desc
</div>
"""

hover_tool = HoverTool(tooltips=tooltip_template, names=['quads'])

# create a new plot with a title and axis labels
fig = figure(plot_height=plot_height, plot_width=plot_width, tools=[hover_tool], min_border=0,
            x_range=(-border_pad,plot_width),y_range=(-plot_height,border_pad), sizing_mode='fixed')
fig.toolbar_location=None
fig.min_border = 0
fig.grid.visible=False
fig.axis.visible=False
fig.outline_line_color=None

# Label column
plotdata = pd.DataFrame({
    'left_edges': np.zeros(nrows),
    'right_edges': np.full(nrows, metric_column_width),
    'top_edges': row_locs,
    'bottom_edges': row_locs-cell_height,
    'cell_fills': np.full(nrows, RGB(255,255,255)),
    'data': metric_column_text,
    'short_desc': metric_column_text,
    'long_desc': metric_descriptions})

right_edges = plotdata['right_edges'].values

for icol,idx in enumerate(metrics.metric_data.index):
    left_edges = right_edges
    right_edges = left_edges + value_column_width
    plotdata = pd.concat((plotdata, pd.DataFrame(
    {'left_edges': left_edges,
    'right_edges': right_edges,
    'top_edges': row_locs,
    'bottom_edges': row_locs-cell_height,
    'cell_fills': [RGB(255,255,255)]+[RGB(*(int(255*x) for x in metric_cmap(metrics.metric_data[metric.key][idx])[:3])) for metric in metrics.metrics],
    'data': ['{}'.format(idx)]+['{:.2f}'.format(metrics.metric_data[metric.key][idx]) for metric in metrics.metrics],
    'short_desc': metric_column_text,
    'long_desc': metric_descriptions})))
    
    #data = 
    #color = 
quads = fig.quad(left='left_edges', right='right_edges', top='top_edges', bottom='bottom_edges',
             source=plotdata, line_color='black', line_width=1, fill_color='cell_fills', name='quads')

fig.text(x='left_edges', y='top_edges', x_offset=left_pad, y_offset=cell_height/2, text='data', source=plotdata, text_baseline='middle', text_font_size='{}pt'.format(fontsize))

# show the results
show(fig)

PyPOP provides built-in functionality to produce a range of commonly desired views of the profiling data.  For example, the calculation of the [POP Metrics](https://pop-coe.eu/node/69) is intended to be a common starting point for all analyses, as they provide an at-a-glance overview of application efficiency and direct the user as to what aspects of the code should be further investigated.  PyPOP has built in support for generating these metrics for various types of parallelism, and outputs colour-coded tables with added contextual (tooltip) information as shown above.

There is also support for more detailed investigation of the program data. Allowing, for example, the visualisation of all OpenMP parallel regions within a code as shown below.  The resultant plot is interactive, allowing the user to explore the execution timeline of the application while providing rich contextual information such as the performance of individual regions, the location within the code and the functions being executed.

This allows the user to rapidly generate interactive and responsive visualisations leveraging the powerful tools and methodology that has been developed by the POP project.

In [5]:
testfile_path="/home/phil/POP/perf_asses/344_gloria/omp_mpi/strong_scaling_8x_instr_r2_c24/8_node/gloria_gcc_ompi.prv"
prv = PRV(testfile_path)

omp_region_stats= prv.profile_openmp_regions()

A Jupyter Widget

In [6]:
metric_palette = diverging_palette(all_palettes['Reds'][256],all_palettes['Greens'][256], 256, 0.8)

tooltip_template = [('Function(s)', "@{Region Function Fingerprint}"),
                    ('Location(s)', "@{Region Location Fingerprint}"),
                    ('Load Bal.', "@{Load Balance}"),
                    ('Length', "@{Region Length} ns"),
                    ('Avg. Comp.', "@{Average Computation Time}"),
                    ('Max. Comp.', "@{Maximum Computation Time}"),
                    ('Sum Comp.', "@{Region Total Computation}"),]

fig = figure(plot_width=900, plot_height=600, tools="xwheel_zoom,zoom_in,zoom_out,pan,reset,save", tooltips=tooltip_template)
for rank, rankdata in omp_region_stats.groupby(level='rank'):
    fig.hbar(y=rank,
             left = 'Region Start',
             right='Region End',
             height = 0.9,
             color=linear_cmap('Load Balance',metric_palette,0,1),
             source=rankdata)
    
show(fig)